In [1]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

#### 1. Choose a reddit page you want to crawl
r/classicalmusic, date range: 8/2/20 - 8/15/20 (2 weeks)

#### 2. The following fields should be present when you crawl

author: author <br />
subreddit: subreddit <br />
date created: created_utc <br />
number of comments: num_comments <br />
score: score <br />
submission title: title <br />
submission description: selftext <br />

In [2]:
def to_utc(date):
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#2 weeks
start_date = dt.datetime.strptime("2020-08-02", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-08-15", "%Y-%m-%d")
 
date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())

sort_type="score"
sort="desc"
fields=["author","subreddit","created_utc","num_comments","score","title","selftext"]
subreddit = 'classicalmusic'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []
for i, s_date in enumerate(date_range):
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 100 #max size 100
        })

        #add logs 
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200: #http status 200 - successful 
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        time.sleep(1)

Doing 2020-08-02 to 2020-08-03
=====Done
Doing 2020-08-03 to 2020-08-04
=====Done
Doing 2020-08-04 to 2020-08-05
=====Done
Doing 2020-08-05 to 2020-08-06
=====Done
Doing 2020-08-06 to 2020-08-07
=====Done
Doing 2020-08-07 to 2020-08-08
=====Done
Doing 2020-08-08 to 2020-08-09
=====Done
Doing 2020-08-09 to 2020-08-10
=====Done
Doing 2020-08-10 to 2020-08-11
=====Done
Doing 2020-08-11 to 2020-08-12
=====Done
Doing 2020-08-12 to 2020-08-13
=====Done
Doing 2020-08-13 to 2020-08-14
=====Done
Doing 2020-08-14 to 2020-08-15
=====Done
Doing 2020-08-15 to 2020-08-16
=====Done


#### 3. After crawling, save your results to a pandas dataframe

In [3]:
flat_list = []
for sublist in results:
    if sublist is not None:
        for item in sublist:
            flat_list.append(item)

#convert from dictionary to dataframe
df = pd.DataFrame.from_dict(flat_list)
#renaming columns to specifications
df.rename(columns={'created_utc':'date created (epoch)','num_comments':'number of comments', 'title':'submission title','selftext':'submission description'},inplace=True)
#creating date column
df['date created'] = pd.to_datetime(df['date created (epoch)'], unit = 's') #convert epoch to readable format
#re odering columns
df = df[['author','subreddit','date created','number of comments','score','submission title','submission description']]
display(df.head())

,author,subreddit,date created,number of comments,score,submission title,submission description
0,art_42069,classicalmusic,2020-08-02 02:07:44,11,1,Please help me find pieces similar to this!!,I am a painter and just started listening to c...
1,lapapinton,classicalmusic,2020-08-02 05:53:02,0,1,Walking shrill: shawm bands in China,
2,kafka84_,classicalmusic,2020-08-02 07:35:08,0,1,"Edvard Grieg - Symphonic Dances, Op. 64 (Paavo...",
3,notwhitebutwong,classicalmusic,2020-08-02 07:32:21,1,1,I'm starting a new series simplifying classica...,
4,prustage,classicalmusic,2020-08-02 09:37:13,1,1,Beethoven being nice: “Wo die Unschuld Blumen ...,


#### 4.a How many submissions were you able to gather?

In [4]:
print(f"There are {len(df)} submissions in total")

There are 905 submissions in total


#### 4.b Who has the most submissions?

***tonevibes has the most submissions at 20 submissions***

In [5]:
df['author'].value_counts() #unique count

tonevibes           20
steinweg            19
boccia45            16
SuperBreakfast      13
lapapinton          11
                    ..
FluidicDegree22      1
Epic_Bunnyhunter     1
SWxNW                1
MasterKeyzPiano      1
GustavMahler9        1
Name: author, Length: 660, dtype: int64

In [6]:
#4.c & 4.d
df.describe()
# max scrore is 2
# max number of comments is 185

,number of comments,score
count,905.000000,905.000000
mean,5.592265,1.001105
std,14.651276,0.033241
min,0.000000,1.000000
25%,0.000000,1.000000
50%,2.000000,1.000000
75%,4.000000,1.000000
max,185.000000,2.000000


#### 4.c Which submission has the highest score?

***comrade_toastboy got the highest score of 2 with the submission title "Where to start?"***

In [7]:
#top 3
df.sort_values('score',ascending=False).head(3)

,author,subreddit,date created,number of comments,score,submission title,submission description
424,comrade_toastboy,classicalmusic,2020-08-09 18:21:16,10,2,Where to start?,"Hi, so I played Violin for about 10 years but ..."
0,art_42069,classicalmusic,2020-08-02 02:07:44,11,1,Please help me find pieces similar to this!!,I am a painter and just started listening to c...
596,delazor,classicalmusic,2020-08-11 10:08:27,34,1,"I made a collage of Beethoven, using that one ...",


#### 4.d Which submission has the highest number of comments?

***jeeruff has the submission with the highest no. of comments at 185 comments with submission title "Principal Piccolo of Berlin Philharmonics, Ask..."***

In [8]:
#top 3
df.sort_values('number of comments',ascending=False).head(3)

,author,subreddit,date created,number of comments,score,submission title,submission description
378,jeeruff,classicalmusic,2020-08-08 16:14:41,185,1,"Principal Piccolo of Berlin Philharmonics, Ask...",[removed]
14,unclemuscles1979,classicalmusic,2020-08-02 15:31:38,141,1,A 1200-1500 piece mint vinyl collection I scor...,
29,Metryco,classicalmusic,2020-08-02 17:33:56,124,1,It's time for some unpopular opinions,I'd just like to see what thoughts listeners a...


#### 4.e Which day of the week has the most submissions?

***for the date range, sunday(6) is the day of the week that has the most submissions at 140***

In [9]:
#dayofweek from a datetime object in pandas. 
#Hint: You may need to use pd.to_datetime to convert your date column...

#creating day of the week column
df['day of the week'] = df['date created'].dt.dayofweek #day of week: 0-monday, 6-sunday
df.groupby('day of the week')['submission title'].count().reset_index().style.highlight_max()

,day of the week,submission title
0,0,139
1,1,130
2,2,120
3,3,124
4,4,130
5,5,122
6,6,140
